In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

# Import our custom modules
import sys
sys.path.append('../../src')

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")


In [ ]:
# Generate synthetic RNA-Seq data
def create_rnaseq_data():
    """Create realistic RNA-Seq expression data."""
    n_samples = 60  # 30 tumor + 30 control
    n_genes = 1000  # Reduced for demonstration
    
    # Create expression matrix with log-normal distribution (realistic for RNA-Seq)
    expression_matrix = np.random.lognormal(mean=5, sigma=1, size=(n_samples, n_genes))
    
    # Add biological structure - differential expression
    tumor_genes = np.random.choice(n_genes, size=100, replace=False)
    control_genes = np.random.choice(n_genes, size=50, replace=False)
    
    # Differential expression patterns
    expression_matrix[:30, tumor_genes] *= 2.5  # Upregulated in tumor
    expression_matrix[30:, control_genes] *= 1.8  # Upregulated in control
    
    # Add batch effects (realistic technical variation)
    batch_effect = np.random.normal(0, 0.3, n_genes)
    expression_matrix[:20] += batch_effect  # Batch 1
    expression_matrix[20:40] += batch_effect * 0.5  # Batch 2
    expression_matrix[40:] += batch_effect * 0.8  # Batch 3
    
    # Create metadata
    sample_labels = ['Tumor'] * 30 + ['Control'] * 30
    batch_labels = ['Batch1'] * 20 + ['Batch2'] * 20 + ['Batch3'] * 20
    
    return expression_matrix, sample_labels, batch_labels, tumor_genes, control_genes

# Create the data
expression_matrix, sample_labels, batch_labels, tumor_genes, control_genes = create_rnaseq_data()

print(f"Expression matrix shape: {expression_matrix.shape}")
print(f"Sample distribution: {pd.Series(sample_labels).value_counts().to_dict()}")
print(f"Batch distribution: {pd.Series(batch_labels).value_counts().to_dict()}")
print(f"Number of tumor-specific genes: {len(tumor_genes)}")
print(f"Number of control-specific genes: {len(control_genes)}")
print(f"Expression range: [{expression_matrix.min():.2f}, {expression_matrix.max():.2f}]")


In [ ]:
def svd_from_scratch(X, n_components=None):
    """
    Implement SVD from scratch using NumPy.
    
    Args:
        X: Input matrix (n_samples x n_features)
        n_components: Number of components to return
        
    Returns:
        U: Left singular vectors (n_samples x n_components)
        S: Singular values (n_components,)
        Vt: Right singular vectors transposed (n_components x n_features)
    """
    # Center the data (important for PCA)
    X_centered = X - X.mean(axis=0)
    
    # Compute covariance matrix (features x features)
    cov_matrix = X_centered.T @ X_centered
    
    # Eigenvalue decomposition of covariance matrix
    eigenvals, V = np.linalg.eigh(cov_matrix)
    
    # Sort by eigenvalues in descending order
    idx = eigenvals.argsort()[::-1]
    eigenvals = eigenvals[idx]
    V = V[:, idx]
    
    # Compute singular values (square root of eigenvalues)
    eigenvals = np.maximum(eigenvals, 0)  # Handle numerical precision
    S = np.sqrt(eigenvals)
    
    # Compute U matrix (left singular vectors)
    U = np.zeros((X_centered.shape[0], len(S)))
    for i in range(len(S)):
        if S[i] > 1e-10:  # Avoid division by very small numbers
            U[:, i] = (X_centered @ V[:, i]) / S[i]
    
    # Remove components with very small singular values
    mask = S > 1e-10
    S = S[mask]
    U = U[:, mask]
    Vt = V[:, mask].T
    
    # Limit to requested number of components
    if n_components is not None:
        n_components = min(n_components, len(S))
        S = S[:n_components]
        U = U[:, :n_components]
        Vt = Vt[:n_components, :]
    
    return U, S, Vt

# Test SVD on expression data
print("Applying SVD to gene expression data...")
U, S, Vt = svd_from_scratch(expression_matrix, n_components=10)

print(f"U shape (samples x components): {U.shape}")
print(f"S shape (singular values): {S.shape}")
print(f"Vt shape (components x genes): {Vt.shape}")
print(f"\\nFirst 10 singular values: {S}")

# Calculate explained variance
total_variance = np.sum((expression_matrix - expression_matrix.mean(axis=0))**2)
explained_variance_ratio = S**2 / total_variance
print(f"\\nExplained variance ratio: {explained_variance_ratio}")
print(f"Cumulative explained variance: {np.cumsum(explained_variance_ratio)}")


In [ ]:
def pca_from_scratch(X, n_components=None):
    """
    Implement PCA from scratch using SVD.
    
    Args:
        X: Input matrix (n_samples x n_features)
        n_components: Number of principal components
        
    Returns:
        X_pca: Transformed data (n_samples x n_components)
        components: Principal components (n_components x n_features)
        explained_variance_ratio: Explained variance ratio for each PC
        mean: Mean of original data
    """
    # Store mean for later use
    mean = X.mean(axis=0)
    
    # Center the data
    X_centered = X - mean
    
    # Apply SVD
    U, S, Vt = svd_from_scratch(X_centered, n_components)
    
    # Transform data to PC space
    X_pca = U @ np.diag(S)
    
    # Principal components are the rows of Vt
    components = Vt
    
    # Calculate explained variance ratio
    total_variance = np.sum(X_centered**2)
    explained_variance_ratio = S**2 / total_variance
    
    return X_pca, components, explained_variance_ratio, mean

# Apply PCA to expression data
print("Applying PCA to gene expression data...")
X_pca, components, explained_variance_ratio, data_mean = pca_from_scratch(expression_matrix, n_components=10)

print(f"PCA transformed data shape: {X_pca.shape}")
print(f"Principal components shape: {components.shape}")
print(f"\\nExplained variance ratio by first 5 PCs:")
for i in range(5):
    print(f"PC{i+1}: {explained_variance_ratio[i]:.4f} ({explained_variance_ratio[i]*100:.2f}%)")

cumulative_variance = np.cumsum(explained_variance_ratio)
print(f"\\nCumulative explained variance:")
for i in range(5):
    print(f"First {i+1} PCs: {cumulative_variance[i]:.4f} ({cumulative_variance[i]*100:.2f}%)")


In [ ]:
# Create comprehensive PCA visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Scree plot
axes[0, 0].plot(range(1, 11), explained_variance_ratio, 'bo-', linewidth=2, markersize=8)
axes[0, 0].set_xlabel('Principal Component')
axes[0, 0].set_ylabel('Explained Variance Ratio')
axes[0, 0].set_title('Scree Plot')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_xticks(range(1, 11))

# 2. Cumulative variance
axes[0, 1].plot(range(1, 11), cumulative_variance, 'ro-', linewidth=2, markersize=8)
axes[0, 1].axhline(y=0.8, color='gray', linestyle='--', alpha=0.7, label='80% threshold')
axes[0, 1].set_xlabel('Principal Component')
axes[0, 1].set_ylabel('Cumulative Explained Variance')
axes[0, 1].set_title('Cumulative Variance Plot')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].set_xticks(range(1, 11))

# 3. PCA scatter plot
colors = ['red' if label == 'Tumor' else 'blue' for label in sample_labels]
axes[1, 0].scatter(X_pca[:, 0], X_pca[:, 1], c=colors, alpha=0.7, s=60)
axes[1, 0].set_xlabel(f'PC1 ({explained_variance_ratio[0]*100:.1f}%)')
axes[1, 0].set_ylabel(f'PC2 ({explained_variance_ratio[1]*100:.1f}%)')
axes[1, 0].set_title('PCA: PC1 vs PC2 (Tumor vs Control)')
axes[1, 0].grid(True, alpha=0.3)

# Add legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='red', label='Tumor'),
                  Patch(facecolor='blue', label='Control')]
axes[1, 0].legend(handles=legend_elements)

# 4. PC1 vs PC3
axes[1, 1].scatter(X_pca[:, 0], X_pca[:, 2], c=colors, alpha=0.7, s=60)
axes[1, 1].set_xlabel(f'PC1 ({explained_variance_ratio[0]*100:.1f}%)')
axes[1, 1].set_ylabel(f'PC3 ({explained_variance_ratio[2]*100:.1f}%)')
axes[1, 1].set_title('PCA: PC1 vs PC3')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Analyze gene loadings (contributions to PCs)
def analyze_pc_loadings(components, n_top_genes=10):
    """Analyze which genes contribute most to each PC."""
    results = {}
    
    for pc_idx in range(min(3, components.shape[0])):
        pc_loadings = components[pc_idx, :]
        
        # Find top positive and negative loadings
        top_positive_idx = np.argsort(pc_loadings)[-n_top_genes:]
        top_negative_idx = np.argsort(pc_loadings)[:n_top_genes]
        
        results[f'PC{pc_idx+1}'] = {
            'top_positive': [(idx, pc_loadings[idx]) for idx in reversed(top_positive_idx)],
            'top_negative': [(idx, pc_loadings[idx]) for idx in top_negative_idx],
            'variance_explained': explained_variance_ratio[pc_idx]
        }
    
    return results

# Analyze loadings
loading_results = analyze_pc_loadings(components, n_top_genes=10)

for pc_name, results in loading_results.items():
    print(f"\\n{pc_name} (explains {results['variance_explained']*100:.2f}% of variance):")
    print("Top positively contributing genes:")
    for gene_idx, loading in results['top_positive'][:5]:
        print(f"  Gene {gene_idx}: loading = {loading:.4f}")
    
    print("Top negatively contributing genes:")
    for gene_idx, loading in results['top_negative'][:5]:
        print(f"  Gene {gene_idx}: loading = {loading:.4f}")

# Check overlap with known gene sets
pc1_loadings = components[0, :]
pc1_top_genes = np.argsort(np.abs(pc1_loadings))[-20:]  # Top 20 by absolute loading

print("\\n" + "="*50)
print("PC1 Gene Set Enrichment Analysis:")
print("="*50)

# Check enrichment for tumor genes
tumor_overlap = len(set(pc1_top_genes) & set(tumor_genes))
tumor_enrichment = (tumor_overlap / len(pc1_top_genes)) / (len(tumor_genes) / expression_matrix.shape[1])

print(f"Tumor-specific gene enrichment:")
print(f"  Overlap: {tumor_overlap}/{len(pc1_top_genes)} genes")
print(f"  Enrichment ratio: {tumor_enrichment:.2f}")

if tumor_enrichment > 2.0:
    print("\\nInterpretation: PC1 is strongly enriched for tumor-specific genes")
    print("PC1 likely captures tumor vs control differences")
else:
    print("\\nInterpretation: PC1 may capture other sources of variation")
